In [8]:
#TextSummarizer

In [2]:
#only for 1st time to install libraries
#! pip3 install nltk

In [2]:

# Importing the required Libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import nltk
#nltk.download('punkt') # one time execution
import re
#nltk.download('stopwords') # one time execution
import matplotlib.pyplot as plt

from nltk.tokenize import sent_tokenize

from nltk.corpus import stopwords

from sklearn.metrics.pairwise import cosine_similarity

import networkx as nx


In [3]:
# Extract word vectors
word_embeddings = {}
file = open('glove.6B.100d.txt', encoding='utf-8')
for line in file:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    word_embeddings[word] = coefs
file.close()
len(word_embeddings)

400000

In [4]:
# reading the file
df = pd.read_excel('test.xlsx')

In [5]:
df

,TEST DATASET,Unnamed: 1
0,NaN,Introduction
1,NaN,AirAsia Aviation Group—controlled by Malaysian...


In [6]:
df.columns

Index(['TEST DATASET', 'Unnamed: 1'], dtype='object')

In [7]:
df.rename(columns = {'Unnamed: 1' : 'Introduction' }, inplace=True)
# Deleting the first row
df.drop(0)

,TEST DATASET,Introduction
1,NaN,AirAsia Aviation Group—controlled by Malaysian...


In [8]:

# Converting the DataFrame into a dictionary
text_dictionary = {}
for i in range(1,len(df['TEST DATASET'])):
    text_dictionary[i] = df['Introduction'][i]
#print(df['Introduction'][1])

In [9]:

# function to remove stopwords
def remove_stopwords(sen):
    stop_words = stopwords.words('english')
    
    sen_new = " ".join([i for i in sen if i not in stop_words])
    return sen_new

In [10]:
# function to make vectors out of the sentences
def sentence_vector_func (sentences_cleaned) : 
    sentence_vector = []
    for i in sentences_cleaned:
        if len(i) != 0:
            v = sum([word_embeddings.get(w, np.zeros((100,))) for w in i.split()])/(len(i.split())+0.001)
        else:
            v = np.zeros((100,))
        sentence_vector.append(v)
    
    return (sentence_vector)


In [11]:
# function to get the summary of the articles
# NOTE - Remove '#' infront of print statement for displaying the contents at different stages of the text summarisation process
def summary_text (test_text, n = 5):
    sentences = []
    
    # tokenising the text 
    sentences.append(sent_tokenize(test_text))
    # print(sentences)
    sentences = [y for x in sentences for y in x] # flatten list
    # print(sentences)
    
    # remove punctuations, numbers and special characters
    clean_sentences = pd.Series(sentences).str.replace("[^a-z A-Z 0-9]", " ")

    # make alphabets lowercase
    clean_sentences = [s.lower() for s in clean_sentences]
    #print(clean_sentences)

    
    # remove stopwords from the sentences
    clean_sentences = [remove_stopwords(r.split()) for r in clean_sentences]
    #print(clean_sentences)
    
    sentence_vectors = sentence_vector_func(clean_sentences)
    
    # similarity matrix
    sim_mat = np.zeros([len(sentences), len(sentences)])
    #print(sim_mat)
    
    # Finding the similarities between the sentences 
    for i in range(len(sentences)):
        for j in range(len(sentences)):
            if i != j:
                sim_mat[i][j] = cosine_similarity(sentence_vectors[i].reshape(1,100), sentence_vectors[j].reshape(1,100))[0,0]
    
    
    nx_graph = nx.from_numpy_array(sim_mat)
    scores = nx.pagerank(nx_graph)
    #print(scores)
    
    ranked_sentences = sorted(((scores[i],s) for i,s in enumerate(sentences)))
    # Extract sentences as the summary
    summarised_string = ''
    for i in range(n):
        
        try:
            summarised_string = summarised_string + str(ranked_sentences[i][1])            
        except IndexError:
            print ("Summary Not Available")
    
    return (summarised_string)


In [12]:
print("Kindly let me know in how many sentences you want the summary - ")
x = int(input())

summary_dictionary = {}

for key in text_dictionary:
    
    para = text_dictionary[key]
    print("Summary of the article - ",key)
    summary = summary_text(para,x)
    summary_dictionary[key] = summary
    
    print(summary)
    print('='*120)    
    
print ("*"*40,"The process has been completed successfully","*"*40)

Kindly let me know in how many sentences you want the summary - 
5
Summary of the article -  1
“Cambodia is a hidden jewel,” he added.AirAsia Cambodia aims to transport inbound tourists from Southeast Asia, China, India and South Korea into the country, which is famous for the ancient Angkor Wat temple in Siem Reap.AirAsia Japan filed for bankruptcy in 2020, while the group divested its stake in an Indian joint venture to partner Tata Sons.The new airline will be led by Cambodian entrepreneur Vissoth Nam as CEO.Fernandes and Kamarudin started AirAsia 21 years ago, with two aircraft.
**************************************** The process has been completed successfully ****************************************


<ipython-input-11-3dbf9a949e2a>:13: FutureWarning: The default value of regex will change from True to False in a future version.
  clean_sentences = pd.Series(sentences).str.replace("[^a-z A-Z 0-9]", " ")


In [20]:


summary_table = pd.DataFrame(list(summary_dictionary.items()),columns = ['TEST DATASET','Summary'])

In [21]:

data_table = pd.DataFrame(list(text_dictionary.items()),columns = ['TEST DATASET','Introduction'])

In [22]:

# Combining the findings into the table
result  = pd.concat([data_table , summary_table['Summary']], axis = 1 , sort = False)
result


,TEST DATASET,Introduction,Summary
0,1,AirAsia Aviation Group—controlled by Malaysian...,"“Cambodia is a hidden jewel,” he added.AirAsia..."


In [23]:

# Saving it to a file (remove the '#' to save)
result.to_csv("testoutput.csv")